### This nodebook creates a report of the count and type of terns exist in the colony. It took Tracked Terns info of a video scan and use a classifier to determine the type of each of tracked tern. Hence we have multiple scans from the same day, we make an aggregation of the different movie scan reports.

In [ ]:
!pip install streamlit==1.31.1 scikit-learn==1.3.2 joblib pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 52.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.8 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
  Attempting uninstall: pillow
    Found existing installation: pillow 11.0.0
    Uninstalling pillow-11.0.0:
      Successfully uninstalled pillow-11.0.0
  Attempting uninstall: packaging
    Found existing installa

### Sets the current working directory to ConvertVideoToImage

In [ ]:
import os

dir = '/content/drive/MyDrive/tern_project/Eyal/ClassifyTerns/'

os.chdir(dir)

### Read dates videos and tracking results dir to convert from config file

In [ ]:
import configparser

# Initialize the ConfigParser object
config = configparser.ConfigParser()
# Read the INI file
config.read('daily_count_terns.ini')
# A date to count terns on
date = config.get('General', 'date')
# Directory path where the tracker result will be located
tracker_result_dir = config.get('General', 'tracker_result_dir')
# Trained classifier model path
classifier_model = config.get('General', 'classifier_model')
# Labels distribution of flags path
labels_distribution_path = config.get('General', 'labels_distribution')

NoSectionError: No section: 'General'

In [ ]:
import os

def get_dir_path_with_parent(directory_path):
  # Get the parent directory and the base name of the directory
  parent_dir, dir_name = os.path.split(os.path.normpath(directory_path))

  # Get the parent directory name and concatenate it with the directory name
  return os.path.join("/", os.path.basename(parent_dir), dir_name)[1:]

Collect all the directories from a specific date

In [ ]:
def get_tour_number(file_path):
    # Split the file path by '/'
    parts = file_path.split('/')
    # Extract the tour number from the last part of the path
    tour_number = parts[-1].split('tour')[-1]
    return tour_number

def get_tour_time(file_path):
    # Split the file path by '/'
    parts = file_path.split('/')
    # Extract the time from the file name
    s = parts[-2].split('_')
    return f'{s[-3]}_{s[-2]}'

In [ ]:
import glob
import datetime

# Tracking resultsdir of all tours scans
# tracker_result_dir = '/content/drive/MyDrive/tern_project/Eyal/TrackingTerns/TrackingResult'


year, month, day = date.split('_')
specific_date = datetime.date(int(year), int(month), int(day))

specific_date_str = specific_date.strftime("%Y_%m_%d")

# Get all directories within tracker_result_dir
tours_dirs_on_date = [dir for dir in glob.glob(os.path.join(tracker_result_dir, '*', '*')) if os.path.isdir(dir) and specific_date_str in dir]

print(tours_dirs_on_date)
grouped_tours_dirs = {}
for dir in tours_dirs_on_date:
    tour_num = get_tour_number(dir)
    tour_time = get_tour_time(dir)
    grouped_tours_dirs.setdefault((tour_time, tour_num), []).append(dir)


grouped_tours_dirs = {key: dirs for key, dirs in grouped_tours_dirs.items() if len(dirs) > 1}
grouped_tours_dirs

['./../TrackingTerns/TrackerResults/OneScanTracks/atlitcam181.stream_2023_06_14_10_00_00/tour0', './../TrackingTerns/TrackerResults/OneScanTracks/atlitcam181.stream_2023_06_14_10_00_00/tour1', './../TrackingTerns/TrackerResults/OneScanTracks/atlitcam181.stream_2023_06_14_15_00_00/tour0', './../TrackingTerns/TrackerResults/OneScanTracks/atlitcam181.stream_2023_06_14_15_00_00/tour1', './../TrackingTerns/TrackerResults/OneScanTracks/atlitcam191.stream_2023_06_14_10_00_00/tour0', './../TrackingTerns/TrackerResults/OneScanTracks/atlitcam191.stream_2023_06_14_10_00_00/tour1', './../TrackingTerns/TrackerResults/OneScanTracks/atlitcam191.stream_2023_06_14_15_00_00/tour0', './../TrackingTerns/TrackerResults/OneScanTracks/atlitcam191.stream_2023_06_14_15_00_00/tour1']


{('10_00',
  '0'): ['./../TrackingTerns/TrackerResults/OneScanTracks/atlitcam181.stream_2023_06_14_10_00_00/tour0', './../TrackingTerns/TrackerResults/OneScanTracks/atlitcam191.stream_2023_06_14_10_00_00/tour0'],
 ('10_00',
  '1'): ['./../TrackingTerns/TrackerResults/OneScanTracks/atlitcam181.stream_2023_06_14_10_00_00/tour1', './../TrackingTerns/TrackerResults/OneScanTracks/atlitcam191.stream_2023_06_14_10_00_00/tour1'],
 ('15_00',
  '0'): ['./../TrackingTerns/TrackerResults/OneScanTracks/atlitcam181.stream_2023_06_14_15_00_00/tour0', './../TrackingTerns/TrackerResults/OneScanTracks/atlitcam191.stream_2023_06_14_15_00_00/tour0'],
 ('15_00',
  '1'): ['./../TrackingTerns/TrackerResults/OneScanTracks/atlitcam181.stream_2023_06_14_15_00_00/tour1', './../TrackingTerns/TrackerResults/OneScanTracks/atlitcam191.stream_2023_06_14_15_00_00/tour1']}

Read statistics distribution of terns type in flags

In [ ]:
import json

# Read labels distribution file
with open(labels_distribution_path, 'r') as json_file:
    labels_distribution_in_flags_json = json.load(json_file)

# Display the first few lines of the JSON content
for i, (key, value) in enumerate(labels_distribution_in_flags_json.items()):
    print(key, value)
    if i == 3:
        break

4 {'Little-Stand': 0, 'Little-Sit': 0.29244808658910104, 'Common-Stand': 0.5669390274895842, 'Common-Sit': 99.14061288592131, 'Common-Fly': 0, 'Little-Fly': 0, 'Common-Stand-Tag': 0, 'Little-Stand-Tag': 0, 'Chick': 0, 'Other': 0}
24 {'Little-Stand': 0, 'Little-Sit': 0.6459420577439776, 'Common-Stand': 5.757713775924624, 'Common-Sit': 93.5963441663314, 'Common-Fly': 0, 'Little-Fly': 0, 'Common-Stand-Tag': 0, 'Little-Stand-Tag': 0, 'Chick': 0, 'Other': 0}
25 {'Little-Stand': 0, 'Little-Sit': 5.851566808555579, 'Common-Stand': 0.15662861081312188, 'Common-Sit': 93.9918045806313, 'Common-Fly': 0, 'Little-Fly': 0, 'Common-Stand-Tag': 0, 'Little-Stand-Tag': 0, 'Chick': 0, 'Other': 0}
26 {'Little-Stand': 0, 'Little-Sit': 16.455371816634585, 'Common-Stand': 0.21135178774633928, 'Common-Sit': 83.33327639561907, 'Common-Fly': 0, 'Little-Fly': 0, 'Common-Stand-Tag': 0, 'Little-Stand-Tag': 0, 'Chick': 0, 'Other': 0}


In [ ]:
def calcAverageCoordinate(boxes_sequence):
    x1, x2, y1, y2 = 0, 0, 0, 0

    for prediction in boxes_sequence:
        x1 += prediction['box']['x1']
        x2 += prediction['box']['x2']
        y1 += prediction['box']['y1']
        y2 += prediction['box']['y2']

    return {'x1': x1 / len(boxes_sequence), 'y1': y1 / len(boxes_sequence), 'x2': x2 / len(boxes_sequence), 'y2': y2 / len(boxes_sequence)}

In [ ]:
def sumClassesPrediction(boxes_sequence):
    classes_prediction = {
        "Little-Stand": {'freq': 0, 'conf': 0},
        "Little-Sit": {'freq': 0, 'conf': 0},
        "Common-Stand": {'freq': 0, 'conf': 0},
        "Common-Sit": {'freq': 0, 'conf': 0},
        "Common-Fly": {'freq': 0, 'conf': 0},
        "Little-Fly": {'freq': 0, 'conf': 0},
        "Common-Stand-Tag": {'freq': 0, 'conf': 0},
        "Little-Stand-Tag": {'freq': 0, 'conf': 0},
        "Chick": {'freq': 0, 'conf': 0},
        "Other": {'freq': 0, 'conf': 0}
    }

    for prediction in boxes_sequence:
        classes_prediction[prediction["name"]]['freq'] += 1
        classes_prediction[prediction["name"]]['conf'] += prediction['confidence']

    for key in classes_prediction.keys():
        classes_prediction[key]['conf'] = (classes_prediction[key]['conf'] / classes_prediction[key]['freq']) if classes_prediction[key]['freq'] > 0 else 0
        classes_prediction[key]['freq'] /= len(boxes_sequence)
        flat_classes_prediction = {}

    for key, value in classes_prediction.items():
        for sub_key, sub_value in value.items():
            new_key = key + '_y_' + sub_key
            flat_classes_prediction[new_key] = sub_value

    return flat_classes_prediction

In [ ]:
import re

def get_flag_number(file_name):
    # Define a regular expression pattern to match the flag number
    pattern = r'flag(\d+).'
    # Use re.search to find the pattern in the file name
    match = re.search(pattern, file_name)
    # Check if a match is found
    if match:
        # Extract the flag number from the matched group
        return match.group(1)
    else:
        print("No flag number found in the file name.")


def get_cam_number(file_name):
    # Define a regular expression pattern to match the flag number
    pattern = r'atlitcam(\d+).'
    # Use re.search to find the pattern in the file name
    match = re.search(pattern, file_name)
    # Check if a match is found
    if match:
        # Extract the flag number from the matched group
        return match.group(1)
    else:
        print("No flag number found in the file name.")

Constructing a dataframe of the boxes sequence details that the classifier needs as features. Each row represent one boxes sequence.

In [ ]:
import json

boxes_details_list = []

for (tour_time, tour_num), tours_dirs in grouped_tours_dirs.items():
    print(tour_time, tour_num)
    print(tours_dirs)

    for tour_dir in tours_dirs:
        # Get the list of files in the directory
        files = [f for f in glob.glob(os.path.join(tour_dir, '*')) if f.endswith('.json')]
        for tracking_on_flag_file in files:
            # Extract the flag number from file name
            flag_number = get_flag_number(tracking_on_flag_file)
            # Extract the camera number from file name
            cam_number = get_cam_number(tracking_on_flag_file)
            # Get tern classes distribution
            flag_labels_distribution = labels_distribution_in_flags_json[flag_number]

            # Open the tracking result JSON file of a flag
            with open(tracking_on_flag_file, 'r') as file:
                tracking_result_report = json.load(file)

            # Shelve tracked objects that detected only few times
            tracked_objects = [tracked_object for tracked_object in tracking_result_report["object_boxes"] if len(tracked_object['predictions']) > 4]

            for tracked_object in tracked_objects:
                boxes_sequence_details = {
                    **(calcAverageCoordinate(tracked_object['predictions'])),
                    **(sumClassesPrediction(tracked_object['predictions'])),
                    **flag_labels_distribution,
                    'detection_rate': len(tracked_object['predictions']) / tracking_result_report['frames_number'],
                    'movement_rate': tracked_object['iou'],
                    'flag': flag_number,
                    'tour_num': tour_num,
                    'tour_time': tour_time,
                    'cam_num': cam_number
                }

                boxes_details_list.append(boxes_sequence_details)


10_00 0
['./../TrackingTerns/TrackerResults/OneScanTracks/atlitcam181.stream_2023_06_14_10_00_00/tour0', './../TrackingTerns/TrackerResults/OneScanTracks/atlitcam191.stream_2023_06_14_10_00_00/tour0']
10_00 1
['./../TrackingTerns/TrackerResults/OneScanTracks/atlitcam181.stream_2023_06_14_10_00_00/tour1', './../TrackingTerns/TrackerResults/OneScanTracks/atlitcam191.stream_2023_06_14_10_00_00/tour1']
15_00 0
['./../TrackingTerns/TrackerResults/OneScanTracks/atlitcam181.stream_2023_06_14_15_00_00/tour0', './../TrackingTerns/TrackerResults/OneScanTracks/atlitcam191.stream_2023_06_14_15_00_00/tour0']
15_00 1
['./../TrackingTerns/TrackerResults/OneScanTracks/atlitcam181.stream_2023_06_14_15_00_00/tour1', './../TrackingTerns/TrackerResults/OneScanTracks/atlitcam191.stream_2023_06_14_15_00_00/tour1']


In [ ]:
import pandas as pd

# Create dataframe for store boxes details
boxes_details_df = pd.DataFrame(boxes_details_list)

boxes_details_df.columns

Index(['x1', 'y1', 'x2', 'y2', 'Little-Stand_y_freq', 'Little-Stand_y_conf',
       'Little-Sit_y_freq', 'Little-Sit_y_conf', 'Common-Stand_y_freq',
       'Common-Stand_y_conf', 'Common-Sit_y_freq', 'Common-Sit_y_conf',
       'Common-Fly_y_freq', 'Common-Fly_y_conf', 'Little-Fly_y_freq',
       'Little-Fly_y_conf', 'Common-Stand-Tag_y_freq',
       'Common-Stand-Tag_y_conf', 'Little-Stand-Tag_y_freq',
       'Little-Stand-Tag_y_conf', 'Chick_y_freq', 'Chick_y_conf',
       'Other_y_freq', 'Other_y_conf', 'Little-Stand', 'Little-Sit',
       'Common-Stand', 'Common-Sit', 'Common-Fly', 'Little-Fly',
       'Common-Stand-Tag', 'Little-Stand-Tag', 'Chick', 'Other',
       'detection_rate', 'movement_rate', 'flag', 'tour_num', 'tour_time',
       'cam_num'],
      dtype='object')

Drop columns of terns class distribution that are not helpful as inputs for our model.


*   Chick
*   Common-Fly
*   Little-Fly
*   Common-Stand-Tag
*   Little-Stand-Tag

Also we change the distribution class names to more informative one

In [ ]:
import pandas as pd

boxes_details_df = boxes_details_df.drop(columns=['Chick', 'Common-Fly','Little-Fly','Common-Stand-Tag', 'Little-Stand-Tag'])

boxes_details_df = boxes_details_df.rename(columns={
    'Little-Stand': 'Little-Stand_f_freq',
    'Little-Sit': 'Little-Sit_f_freq',
    'Common-Stand': 'Common-Stand_f_freq',
    'Common-Sit': 'Common-Sit_f_freq',
    'Other': 'Other_f_freq',
})

Calculating for each box the width and heigh size in cm. The calculations involve camera calibration, 3D to 2D transformations, and the use of PTZ (Pan-Tilt-Zoom) camera parameters. It's done by using a library writted to mapping box pixel coordinates into real world location.

In [ ]:
import sys

sys.path.append('/content/drive/MyDrive/tern_project/Eyal/RealCoordinatesCalculator')
from real_coordinates_calculator import RealCoordinatesCalculator


real_coordinates_calculator = RealCoordinatesCalculator()
# Group by 'flag' and apply the box size calculation function
boxes_size_df = boxes_details_df.groupby('flag').apply(real_coordinates_calculator.calc_box_size)

# Reset the index for later merging
boxes_size_df.reset_index(drop=True, inplace=True)

# Merge the calculated results back into the original DataFrame
boxes_details_df = pd.merge(boxes_details_df, boxes_size_df, left_index=True, right_index=True)

# Display the resulting DataFrame
boxes_details_df.head()

Loading file /content/drive/MyDrive/tern_project/Eyal//PTZCamValues181_mod.txt...
Loading file /content/drive/MyDrive/tern_project/Eyal//PTZCamValues191_mod.txt...


<ipython-input-14-1ccf43c8e287>:9: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  boxes_size_df = boxes_details_df.groupby('flag').apply(real_coordinates_calculator.calc_box_size)


,x1,y1,x2,y2,Little-Stand_y_freq,Little-Stand_y_conf,Little-Sit_y_freq,Little-Sit_y_conf,Common-Stand_y_freq,Common-Stand_y_conf,...,dy/dx_uv,pix_x1,pix_y1,pix_x2,pix_y2,DX_cm,DY_cm,dx_pix_drone,dy_pix_drone,Area
0,342.825964,205.914452,402.556092,239.921489,0.0,0.0,0.0,0.0,1.0,0.689080,...,0.554398,9696,13578,9839,13539,79.794197,44.237738,143,39,3529.914748
1,892.188696,316.144509,935.644333,349.254809,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.891064,7921,13604,8148,13576,125.837575,112.129290,227,28,14110.078013
2,740.807082,238.721267,799.854833,276.696790,0.0,0.0,0.0,0.0,1.0,0.633201,...,1.132896,9567,13371,9747,13356,100.196240,113.511870,180,15,11373.462560
3,665.280541,252.666935,715.494240,287.265885,0.0,0.0,0.0,0.0,1.0,0.640355,...,1.090384,7957,13369,8178,13356,122.595105,133.675768,221,13,16387.994799
4,663.995733,300.748086,692.409661,330.062228,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.937865,9622,13505,9675,13498,29.527487,27.692809,53,7,817.699038


## Classify all sequences of boxes representation by a pre-trained Random Forest model.

#### Load pre-trained Random Forest model

In [ ]:
import joblib

# Load the trained Random Forest model
rf_model = joblib.load(classifier_model)

In [ ]:
# Re-order feature to fit with the models features
X = boxes_details_df[rf_model.feature_names_in_]

In [ ]:
y_pred = rf_model.predict(X)

boxes_details_df['y_pred'] = y_pred

### Aggregate 191 and 181 cameras results for each tour scan seperately.

In [ ]:
# Get unique combinations of values in the two categorical columns
unique_combinations = boxes_details_df.groupby(['tour_time', 'tour_num']).size().reset_index()

# Create a dictionary to hold DataFrames for each combination
combination_dfs = {}

# Split the DataFrame based on the unique combinations of the categorical columns
for index, row in unique_combinations.iterrows():
    tour_num = row['tour_num']
    tour_time = row['tour_time']
    combination_dfs[(tour_num, tour_time)] = boxes_details_df[(boxes_details_df['tour_num'] == tour_num) & (boxes_details_df['tour_time'] == tour_time)]


In [ ]:
for tour_num, tour_time in combination_dfs.keys():
    print(f'Result from tour {tour_num} at {tour_time}:')
    print(combination_dfs[(tour_num, tour_time)]['y_pred'].value_counts())
    print()

Result from tour 0 at 10_00:
y_pred
Common-Sit      462
Little-Sit      151
Common-Stand     80
Background       19
Chick             4
Name: count, dtype: int64

Result from tour 1 at 10_00:
y_pred
Common-Sit      492
Little-Sit      126
Common-Stand     99
Background       27
Chick             7
Little-Stand      2
Name: count, dtype: int64

Result from tour 0 at 15_00:
y_pred
Common-Sit      451
Little-Sit      117
Common-Stand     94
Background       15
Chick            12
Name: count, dtype: int64

Result from tour 1 at 15_00:
y_pred
Common-Sit      498
Common-Stand     97
Little-Sit       85
Background       21
Chick            12
Name: count, dtype: int64



In [ ]:
terns_details_on_tour = combination_dfs[('0', '10_00')]

#### Display the terns locations on the drone map

In [ ]:
import cv2

## The drone image
drone_img = cv2.imread('/content/drive/MyDrive/tern_project/drone_rotem/Shafiyot_True_Ortho.tif')

In [ ]:
from google.colab.patches import cv2_imshow

for index, row in terns_details_on_tour.iterrows():
    if 'Common' in row['y_pred']:
        cv2.circle(drone_img, [row['pix_x1'], row['pix_y1']], radius=25, color=(0,0,255), thickness=-1)
    elif 'Little' in row['y_pred']:
        cv2.circle(drone_img, [row['pix_x1'], row['pix_y1']], radius=25, color=(0,255,255), thickness=-1)
    else:
        cv2.circle(drone_img, [row['pix_x1'], row['pix_y1']], radius=25, color=(255,0,0), thickness=-1)


# Resize the image
scale_factor = 0.1
resized_image = cv2.resize(drone_img, (0, 0), fx=scale_factor, fy=scale_factor)

cv2_imshow(resized_image)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
from google.colab.patches import cv2_imshow


## I did for one example - one flag.
## with the final results and the right categories, we can do it for all of the flags and then decide how to treat overlapping areas.

# for index, row in df_ptz_modi[df_ptz_modi['ptz_num']==24].iterrows():
#       pitch = row['pitch']
#       yaw = row['yaw']
#       f = row['f']
#       ptz_num = row ['ptz_num']


#       r = R.from_euler('zyx', [yaw, 0, pitch],degrees=True).as_matrix()
#       k = make_k(f, center)

#       ## the location in cm in the north camera coordinate
#       ## here you always use the location of the north camera (cam_n)
#       p = calc_p(df,r,k)


#       ## calculate the terns locations in pixel of the drone
#       ## when calculate in pixels, in the south camera we need to use te south camera location (south_cam_loc)

#       tern_loc_drone = tern_loc_pix(s,north_cam_loc, p)

for i, row in df.iterrows():
    # print(row['pix_x1'])
    tern_x,tern_y = row['pix_x1'], 	row['pix_y1']
    cv2.circle(drone_img, [tern_x, tern_y], radius=25, color=(0,0,255), thickness=-1)

# Resize the image
scale_factor = 0.1


resized_image = cv2.resize(drone_img, (0, 0), fx=scale_factor, fy=scale_factor)


cv2_imshow(resized_image)




NameError: name 'df' is not defined